#### Etapa 2 - Processo de Segmentação
#### (Definição de Classes: Marrom, Amarela e Verde)

In [1]:
## Importação de Bibliotecas

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
from sklearn.cluster import KMeans
import skimage.color
import cv2
from scipy import misc
from skimage.color import rgb2gray
from skimage import color
import skimage.metrics
from sklearn.metrics import mean_squared_error
from skimage.metrics import structural_similarity
from skimage.metrics import peak_signal_noise_ratio
from PIL import Image
from scipy import ndimage
import time
import pandas as pd
import statistics as st
import glob
from IPython.display import display, Javascript
import seaborn as sns
import os
from IPython.display import display, Javascript
from os import chdir, getcwd, listdir
from os.path import isfile
import oci
import boto3
import functions as fun

### ------------------------------------ ###
### Inicialização do contador de imagens
### ------------------------------------ ###
global contador_imagens
contador_imagens =66

### ------------------------------------ ###
### Inicialização da Cor do Processamento
### 1- Verde, 2- Amarela e 3- Marrom
### ------------------------------------ ###
global cor
cor =3

In [ ]:
#####---------------------------------------#####
 ## PASSOS APÓS A EXECUÇÃO DA PRIMEIRA IMAGEM ##
#####---------------------------------------#####

## Função para parâmetrização para execução do algoritmo
def parametrizacao(contador_imagens):
           
    # Parâmetro de Data/Hora para gravar no Banco de Dados
    global dataHora
    dataHora = time.strftime('%Y-%m-%d %H:%M:%S')

    # Parâmetros da Janela do Objeto de Interesse (Coordenadas)
    # --> Definidos na Etapa 1
    # --> Valores de Janela (ROI): 
                            # 600, 2700, 1250, 2800 (imagens que apresentar erro "out of baunds" se fragmento) 
                            # 550, 2700, 700, 3200 (Todas as imagens)
    coordJanela_1, coordJanela_2, coordJanela_3, coordJanela_4 = 900, 2700, 700, 3200
    
    global param_coord_janela_Obj_int
    param_coord_janela_Obj_int = coordJanela_1, coordJanela_2, coordJanela_3, coordJanela_4

    ## Definições Gerais
    global tipoTeste
    tipoTeste = 'Imagem real DB público Embrapa'

    # Parâmetros de Imagem
    global tipoImgOriginal
    tipoImgOriginal = 'RGB'
    
    return coordJanela_1,coordJanela_2, coordJanela_3, coordJanela_4, param_coord_janela_Obj_int,\
tipoTeste, dataHora, tipoImgOriginal, contador_imagens

## Chamada de função para parâmetrização para execução do algoritmo
coordJanela_1,coordJanela_2, coordJanela_3, coordJanela_4, param_coord_janela_Obj_int,\
    tipoTeste, dataHora, tipoImgOriginal, contador_imagens = parametrizacao(contador_imagens)

## Imprimir o contador de imagens
print('\nContador de imagens: ' + str(contador_imagens))

#### Configurações e Funcionalidades Buckets - Oracle Cloud

In [3]:
# Função para configurações da Cloud
def config_cloud(flag):

    # Definição de variável URL de acesso ao Endpoint Oracle Cloud
    if flag == 'p':
        # Ambiente de Produção - Config Cloud
        url = "https://grinqewrovfi.compat.objectstorage.sa-saopaulo-1.oraclecloud.com"
    
    elif flag =='t':
        
        # Ambiente de Testes - Config Cloud
        url = "https://grwbzp0j0zza.compat.objectstorage.sa-saopaulo-1.oraclecloud.com"

    # Chamada de função para obter as configurações da Cloud
    fun.configuration_Cloud()

    # Chamada de função para obter o resource
    resource_value = fun.resource()
       
    return url, resource_value

# Chamada de função para configurar a Cloud 
url, resource_value = config_cloud('p')


In [ ]:
# Função para pegar os objetos selecionados no bucket
def objects_to_bucket(flag, bucket, resource_value, url, contador_imagens):
    # Definição do Path
    path_in = "segmented-images-1/"
    
    if os.path.exists(path_in)==False:
        os.makedirs(path_in)
        
    # Imprimir os buckets encontrados
    try:
        for bucket_name in resource_value.buckets.all():
            print (bucket_name.name)
        print("\n Conexao com sucesso na cloud!!\n")
    except:
        print("Falha de conexao na cloud")    
    
    files = fun.getObjectsToBucket(str(bucket), resource_value)

    # Flag 'd' - Processo de download de dados via (API) Boto3 
    if flag == 'd':
        # Function call for download file: destination data science workspace - folder db_imagens
        fun.download_to_bucket(str(bucket), url, files, path_in)
        message = "Download de arquivos com sucesso!!"
    
    # Flag 's' - Processo de sincronismo de dados via (API) rclone
    elif flag == 's':
        try:
            #rclone sync source:path dest:path
            cmd = 'rclone sync ' + str(bucket) + ':' + str(bucket) + ' ' + path_in 
            os.system(cmd)
            message = "Arquivos sincronizados com successo!!"
        except:
            message = "Erro de sincronismo"
    
    return message, path_in

# Chamada de função para obter objetos selecionados do bucket
message, path_in = objects_to_bucket('s', 'BK-segmented-images-selected', resource_value, url, contador_imagens)
print(message)


In [ ]:
# Função para armazenar Filepaths para processamento
def armazenar_filepaths_processamento(contador_imagens):
    global nome_imagem
    # Configurar parâmetros dos diretórios
    path_in = 'segmented-images-1/'
          
    filepaths = [f for f in os.listdir(path_in) if f.endswith('.jpg')]
    number_files = len(filepaths)

    nome_imagem = str(filepaths[contador_imagens])
    pasta = nome_imagem[-12::]
    nomeTeste= 'Processamento '+ str(contador_imagens) + ' - Qualificação'
    
    return path_in, nome_imagem, pasta, filepaths, nomeTeste, number_files

# Chamada de função para armazenar Filepaths para processamento
path_in, nome_imagem, pasta, filepaths, nomeTeste, number_files = armazenar_filepaths_processamento(contador_imagens)
filepaths

#### Armazenar as imagens recebidas do Bucket para processamento

In [6]:
# Função para as definições dos parâmetros da cor (1: Verde, 2: Amarela, 3: Marrom)
def definicao_cores(nome_imagem, cor):
    global classe_cor
    global corRGBSemente
    
    # Verifica a "classe" da imagem selecionada (Classe Verde = 1 | Classe Amarela = 2 | Classe Marrom = 3)
    if cor ==1:
        corRGBSemente = '75,104,29' # Pixel Verde
        classe_cor = 'verde'
        # Parâmetro de janela: 8 (classe verde)
        pixels_reta = 8
        corPixelReferencia = 104
                
    elif cor ==2:
        corRGBSemente = '153,137,050' # Pixel Amarelo
        classe_cor = 'amarela'
        # Parâmetro de janela: 7 (classes - marrom e amarela)
        pixels_reta = 7
        corPixelReferencia = 137
        
    elif cor ==3:
        corRGBSemente = '109,75,38' # Pixel Marrom
        classe_cor = 'marrom'
        # Parâmetro de janela: 7 (classes - marrom e amarela)
        pixels_reta = 7
        # Parâmetros de Pixels de Referência
        corPixelReferencia = 75
        
    # Estruturação dos diretórios
    path_out = 'Segmentacao_Etapa_2/' + pasta + '/' + classe_cor
    path_out_labels = path_out + '/labels/'
    os.makedirs(path_out)
    os.makedirs(path_out_labels)
       
    return classe_cor, corRGBSemente, corPixelReferencia, pixels_reta, path_out, path_out_labels
           
# Chamada de função para definição de cores da imagem
classe_cor, corRGBSemente, corPixelReferencia, pixels_reta, path_out,\
    path_out_labels = definicao_cores(nome_imagem, cor) 

In [ ]:
# Função para Leitura do arquivo de Imagem no disco
def ler_arquivo_imagem(nome_imagem):
    global img_orig
    imagemOriginal = path_in + nome_imagem
    img_orig = plt.imread(imagemOriginal)

    #Exibição da Imagem
    plt.figure(figsize=[10,10])
    plt.imshow(img_orig)

    # Chamada de função para gravar em Disco a imagem
    fun.salvarImagemDiscoRGB(img_orig, path_out + '/original_' + nome_imagem)    

    return img_orig

# Chamada de função para Leitura do arquivo de Imagem no disco
img_orig = ler_arquivo_imagem(filepaths[contador_imagens])

In [ ]:
# Chamar função de exibição de propriedades da imagem
# Obter as propriedades detalhadas da Imagem
larguraImgOriginal, alturaImgOriginal, numCanaisImgOriginal, totalPixelsImgOriginal = fun.caracteristicasImagem(img_orig)

In [ ]:
# Chamada de função para plotar imagens com os canais de cores separados
# Plotar imagem dividida por canais (RGB)
global img_green
img_red, img_green, img_blue = fun.plotaGraficoCanaisImagem(img_orig)

#### Salvar a imagem do Canal Verde em disco

In [ ]:
# Função para gravar em Disco Imagem do Canal Verde
def gravar_imagem_canal_verde(path_out, classe_cor, nome_imagem, img_green):
    imgCanalVerde = path_out + '/img_canal_verde_' + classe_cor + '_' + nome_imagem
    fun.salvarImagemSegmentadaDisco (img_green, imgCanalVerde)
    # Function call Upload files function from workspace files to Selected Bucket
    fun.upload_from_bucket('BK-segmented-images-2', url, path_out + '/img_canal_verde_' + classe_cor + '_' + nome_imagem)

    #Plotar a imagem
    plt.figure(figsize=[6, 6])
    plt.imshow(img_green)

# Chamada de função para gravar em Disco
gravar_imagem_canal_verde(path_out, classe_cor, nome_imagem, img_green)


#### Coleta Automática da melhor Semente - Ponto Central Automatização
#### Análise de vizinhança em cada pixel da imagem

In [ ]:
# Função para encontrar, automaticamente, o Pixel semente (com base no cálculo da vizinhança dos Pixels)
def encontrar_pixel_semente(img_green,corPixelReferencia, coordJanela_1, coordJanela_2, coordJanela_3,\
     coordJanela_4, url, path_out, nome_imagem, classe_cor):
     
    # Ler a imagem (Canal Verde) na região de interesse
    coordenada_x, coordenada_y, erro, text = fun.calcularVizinhanca(img_green,corPixelReferencia,\
         coordJanela_1, coordJanela_2, coordJanela_3, coordJanela_4)
    
    global coleta_aut_semente, coordenadasSementeCentral
    coleta_aut_semente = text

    coordenadasSementeCentral = str(coordenada_x) + " , " + str(coordenada_y)

    # Gravar o resultado do processamento em arquivo
    with open(path_out + '/dados_coleta_autom_semente_' + nome_imagem + '_' + classe_cor + '.txt', 'w') as f:
        f.write(coleta_aut_semente)

    # Function call Upload files function from workspace files to Selected Bucket
    fun.upload_from_bucket('BK-segmented-images-2', url, path_out + '/dados_coleta_autom_semente_' +\
         nome_imagem + '_' + classe_cor + '.txt')

    # imprimir na tela os resultados finais do processamento 
    print(coleta_aut_semente)

    return coordenada_x, coordenada_y, erro, coleta_aut_semente, coordenadasSementeCentral

# Chamada de função para encontrar, automaticamente, o Pixel semente (com base no cálculo da vizinhança dos Pixels)
coordenada_x, coordenada_y, erro, coleta_aut_semente,coordenadasSementeCentral\
    = encontrar_pixel_semente(img_green,corPixelReferencia, coordJanela_1, coordJanela_2,\
                              coordJanela_3, coordJanela_4, url, path_out, nome_imagem, classe_cor)


#### Coleta de Pixels a partir da semente como Ponto Central (Coordenadas obtidas no passo anterior)
#### Uma Janela de Pixels será gerada para cada Cor 

In [ ]:
# Função para calcular janela de pixels para a população
def calcular_janela_pixels(coordenada_x, coordenada_y, pixels_reta, img_green, path_out, url, nome_imagem, classe_cor):

    # Informar o ponto (coordenada x e y) correspondente à semente de interesse (ponto de centro)
    # configuração da função: parâmetro 1: coordenada x; parâmetro 2: coordenada y; parâmetro 3: tamanho da janela em pixels a partir do centro

    lin_val_1, lin_val_2, col_val_1, col_val_2 = fun.calcularJanela(coordenada_x,coordenada_y,pixels_reta)

    # Imprimir as coordenadas de janela calculadas
    global coordenadasCalcJanela, janela, totalPixelsJanela
    coordenadasCalcJanela = (lin_val_2, lin_val_1, col_val_1, col_val_2)
    print(coordenadasCalcJanela)

    # Processa a construção da janela de pixels
    janela = np.zeros_like(img_green)
    lista_pixels = []

    for col in range(col_val_2, col_val_1):
        for lin in range(lin_val_1,lin_val_2):
            janela[col,lin] = img_green[col,lin]
            lista_pixels.append(janela[col,lin])

    # Gravar os dados da imagem em arquivo .CSV
    fun.gravarArquivoCSV(lista_pixels, path_out + '/dados_pixels_janela_' + nome_imagem +\
                         '_' + classe_cor + '.csv')

    # Chamada de função para upload de arquivos da 'pasta de trabalho' para o Bucket selecionado
    fun.upload_from_bucket('BK-segmented-images-2', url, path_out + '/dados_pixels_janela_' +\
                           nome_imagem + '_' + classe_cor + '.csv')

    # Exibir a quantidade de pixels da janela de pixels
    totalPixelsJanela = (lin_val_2 - lin_val_1)*(col_val_1 - col_val_2)
    print(str(lin_val_2 - lin_val_1) + " X " + str(col_val_1 - col_val_2) + " = " +
        str(totalPixelsJanela) + " Pixels")

    # Exibir a imagem da janela
    plt.figure(figsize=[10, 10])
    plt.imshow(janela)

    # Gravar em disco a imagem com a/janela
    imgJanela = path_out + '/img_janela_' + classe_cor + '_' + nome_imagem
    fun.salvarImagemDiscoRGB(janela, imgJanela)

    # Chamada de função para upload de arquivos da 'pasta de trabalho' para o Bucket selecionado
    fun.upload_from_bucket('BK-segmented-images-2', url, path_out + '/img_janela_' +\
                           classe_cor + '_' + nome_imagem)

    return lin_val_1, lin_val_2, col_val_1, col_val_2, janela, imgJanela, lista_pixels,\
            coordenadasCalcJanela, totalPixelsJanela

# Chamada de função para calcular janela de pixels para a população
lin_val_1, lin_val_2, col_val_1, col_val_2, janela, imgJanela, lista_pixels, coordenadasCalcJanela,\
    totalPixelsJanela = calcular_janela_pixels(coordenada_x, coordenada_y, pixels_reta, img_green, \
                                               path_out, url, nome_imagem, classe_cor)


#### Mostra os dados dos pixels da Janela

In [13]:
# Função para carregar os dados CSV no Dataframe "Transposto"
def carregar_dados_dataframe_transposto(path_out, nome_imagem, classe_cor):
    df_pixels = pd.read_csv(path_out + '/dados_pixels_janela_' + nome_imagem + '_' + classe_cor + '.csv', header= None).T

    # Atribuição dos valores da janela para uso como sementes
    # Conversão da lista de strings para lista de inteiros
    sementes = list(map(int, lista_pixels))

    return df_pixels, sementes

# Chamada de função para carregar os dados CSV no Dataframe "Transposto"
df_pixels, sementes = carregar_dados_dataframe_transposto(path_out, nome_imagem, classe_cor)

#### Gráfico Boxplot
#### Foco na visualização da mediana

In [ ]:
# Função para gerar o gráfico Bloxplot dos dados da Janela
def gerar_bloxpot(lista_pixels, path_out, classe_cor, nome_imagem, url, string):
    sns.boxplot(data=lista_pixels)
    # Gravar em disco a imagem com a janela
    plt.savefig(path_out + string + classe_cor + '_' + nome_imagem, format='png')
    plt.show()
    # Function call Upload files function from workspace files to Selected Bucket
    fun.upload_from_bucket('BK-segmented-images-2', url, path_out + string + classe_cor + '_' + nome_imagem)

# Função para Calcular os valores do Gráfico Boxplot
def calcular_dados_boxplot(dados):
    # Considerar a (constante) = 1.5
    constante = 1.5
    boxplot = ''
    
    # Calcular Primeiro Quartil (q1)
    q1=np.percentile(dados,25)
    # Calcular Terceiro Quartil (q3)
    q3=np.percentile(dados,75)
    # Calcular Intervalo Interquartil (intervalo_interq)
    intervalo_interq=q3-q1
    # Calcular o limite Inferior       
    limite_inferior=q1 - constante*intervalo_interq
    # Calcular o limite Superior     
    limite_superior=q3 + constante*intervalo_interq
    
    # Amrazenar os resultados em arquivo.txt
    boxplot += "================================================================= \n"
    boxplot += "Cálculos Boxplot \n"
    boxplot += "================================================================= \n"
    boxplot += "Primeiro Quartil (Q1): " +str(q1) + "\n"
    boxplot += "Terceiro Quartil (Q3): " +str(q3) + "\n"
    boxplot += "Limite Inferior (LI): "  +str(limite_inferior) + "\n"
    boxplot += "Limite Superior (LS): "  +str(limite_superior) + "\n"
    boxplot += "================================================================= \n"
    
    return q1, q3, limite_inferior, limite_superior, boxplot 
    
# Função para Carregar os valores de Outliers e sem Outliers nas listas
def carregar_dados_outliers(dados, q1, q3, limite_inferior, limite_superior):
    # Listas de Resultados
    lista_outliers= []
    lista_sem_outliers= []
    
    for d in dados:
        if (d > limite_superior) or (d < limite_inferior):
            lista_outliers.append(d)
        else:
            lista_sem_outliers.append(d)
    
    return lista_outliers, lista_sem_outliers   

# Chamada de função para gerar o gráfico Bloxplot dos dados da Janela
gerar_bloxpot(lista_pixels, path_out, classe_cor, nome_imagem, url, '/img_boxplot_dados_janela_')

# Chamada de Função para Calcular os valores do Gráfico Boxplot
q1, q3, limite_inferior, limite_superior, boxplot=calcular_dados_boxplot(lista_pixels)

# Chamada de Função para Carregar os valores de Outliers e sem Outliers nas listas
lista_outliers, lista_sem_outliers =carregar_dados_outliers(lista_pixels, q1, q3, limite_inferior, limite_superior)

#### Cálculo estatístico sementes Classes (Verde, amarela e marrom)
#### Configuração dos limiares 1 e 2 - considerando a MEDIANA dos dados de pixels sementes (coletados de forma supervisionada ou automatizada)
#### Executados por classe (Verde, amarelo e marrom)

In [ ]:
# Função para cálculo de limiares
# Configuração dos limiares 1 e 2 - considerando a variância dos dados de pixels sementes
# (coletados de forma supervisionada) - Executados por classe (Verde, amarelo e marrom)

# Uso de sementes calculadas a partir do uso da "Janela de Pixels"

# Parâmetros de entrada: Vetor semente (verde ou amarelo ou marrom),
                       # Variável operação: 1: variância polulacional; 2:variância amostral,
                                         #  3:média aritmética, 4:mediana, 5:média ponderada
                       # valor do sigma (1 ou 2)

def calcular_limiares(nome_imagem, classe_cor):

    th1, th2, dp, op, var, valor = fun.calcularLimiares(sementes,4,1)
    global range_value, dados_estatisticos, totalSementesCalculadas
    range_value = [th1, int(th1+(th2-th1)/2), th2]

    # Calcular o erro
    erro = (dp*100)/valor

    dados_estatisticos =''
    dados_estatisticos+= "Classe: " + classe_cor + " - Imagem: " + nome_imagem + "\n"
    dados_estatisticos+= "\n Sementes da Janela: " + str(sementes) + "\n Variância: " +\
        str(var) + "\n Limiar 1: " + str(th1) + "\n Limiar 2: " + str(th2) + \
    "\n Faixa Limiares: " + str(range_value) + "\n Desvio Padrão (Raiz quadrada variância): " \
    + str(dp) + "\n Operação: " + op + " = " + str(valor) + "\n"
    dados_estatisticos+= " Contagem de sementes: " + str(len(sementes)) + "\n"
    dados_estatisticos+= " Erro calculado: " + str(erro) + "\n"
        
    # Vetor auxiliar para armazenar os outliers (limite inferior)
    lim_inferior_outliers = []
    # Vetor auxiliar para armazenar os outliers (limite superior) 
    lim_superior_outliers = []
    num_repeticoes = 0

    # Enquanto o erro for maior que 5% retirar os outliers e recalcular
    while (erro >=5):
        for i in sementes:
            if i < valor - dp:
                lim_inferior_outliers.append(i)
            if i > valor + dp:
                lim_superior_outliers.append(i)

            for i in lim_inferior_outliers:
                for j in sementes:
                    if j == i:
                        sementes.remove(j)

            for i in lim_superior_outliers:
                for j in sementes:
                    if j == i:
                        sementes.remove(j)
        
        th1, th2, dp, op, var, valor = fun.calcularLimiares(sementes,4,1)
        range_value = [th1, int(th1+(th2-th1)/2), th2]
        num_repeticoes+=1

        # Recalcular o erro (condição de parada)
        erro = (dp*100)/valor
        dados_estatisticos = imprimir(th1, th2, dp, op, var, valor, erro,\
                                      range_value, num_repeticoes, lim_inferior_outliers, lim_superior_outliers)
    
    totalSementesCalculadas = len(sementes)
    # Imprimir o resultado na tela
    print(dados_estatisticos)

    # Grava a String em arquivo
    with open(path_out + '/dados_calculo_estatistico_' + nome_imagem + '_' + classe_cor + '.txt', 'w') as f:
        f.write(dados_estatisticos)

    # Chamada de função para upload de arquivos da 'pasta de trabalho' para o Bucket selecionado
    fun.upload_from_bucket('BK-segmented-images-2', url, path_out + '/dados_calculo_estatistico_' + nome_imagem + '_' + classe_cor + '.txt')

    return th1, th2, dp, op, var, valor, totalSementesCalculadas, dados_estatisticos, erro, range_value, num_repeticoes, lim_inferior_outliers, lim_superior_outliers

# Função para armazenar informações dos cálculos efetuados durante o processo
def imprimir(th1, th2, dp, op, var, valor, erro, range_value, num_repeticoes, lim_inferior_outliers, lim_superior_outliers):
    text = ''
    text += "================================================================= \n"
    text += "\n Sementes calculadas " +str(num_repeticoes) + ": " + str(sementes) + "\n Variância: " + str(var) + "\n Limiar 1: " + str(th1) + \
    "\n Limiar 2: " + str(th2) + "\n Faixa Limiares: " + str(range_value) + "\n Desvio Padrão (Raiz quadrada variância): " \
    + str(dp) + "\n Operação: " + op + " = " + str(valor) + "\n"
    text += " Contagem de sementes: " + str(len(sementes)) + "\n"
    text += " Limite inferior: " + str(len(lim_inferior_outliers)) + "\n"
    text += " Limite superior: " + str(len(lim_superior_outliers)) + "\n"
    text += " Erro calculado: " + str(erro) + "\n"
    text += " Número da iteração: " + str(num_repeticoes) + "\n"
    return text

# Chamada de Função para cálculo de limiares
th1, th2, dp, op, var, valor, totalSementesCalculadas, dados_estatisticos, erro, range_value, num_repeticoes,\
    lim_inferior_outliers, lim_superior_outliers = calcular_limiares(nome_imagem, classe_cor)


In [ ]:
# Exibir o gráfico Bloxplot dos dados da Janela
# Chamada de função para gerar o gráfico Bloxplot dos dados da Janela
### Visualização Pós Cálculo Estatístico e corte de Limites (inferior e superior) 
gerar_bloxpot(sementes, path_out, classe_cor, nome_imagem, url, '/img_boxplot_dados_pos_calculo_estatistico_')

# Função para Calcular os valores Finais do segundo Gráfico Boxplot
def finalizar_calculo_boxplots(dados, lista_outliers, lista_sem_outliers, boxplot):
    # Chamada de Função para Calcular os valores do Gráfico Boxplot
    q1, q3, limite_inferior, limite_superior, boxplot_2=calcular_dados_boxplot(dados)
    
    # Chamada de Função para Carregar os valores de Outliers e sem Outliers nas listas
    lista_outliers_2, lista_sem_outliers_2 =carregar_dados_outliers(dados, q1, q3, limite_inferior, limite_superior)
    
    # contagem das listas - Pré-Cálculo (pixels)
    contagem_lista_sem_outliers=len(lista_sem_outliers)
    contagem_lista_outliers=len(lista_outliers)
    
    # contagem das listas - Pós-Cálculo (pixels)
    contagem_lista_sem_outliers_2=len(lista_sem_outliers_2)
    contagem_lista_outliers_2=len(lista_outliers_2)
    
    # Contabilizar Contagem Sem Outliers
    sem_outliers=contagem_lista_sem_outliers + contagem_lista_sem_outliers_2
    
    # Contabilizar soma Outliers
    outliers=contagem_lista_outliers + contagem_lista_outliers_2           
            
    # Unir dados de Cálculos dos Boxplots ("texto")
    texto = ""
    texto += "==================== Boxplot - Pré-Cálculo ====================== \n\n"
    texto += boxplot
    texto += "\n==================== Boxplot - Pós-Cálculo ====================== \n\n"
    texto += boxplot_2
    texto += "                     Resultados Contabilizados \n"
    texto += "=================================================================\n"    
    texto += "Contagem Individual Outliers -> (Pré-Cálculo: " + str(contagem_lista_outliers) + ")\n"
    texto += "Contagem Individual Outliers -> (Pós-Cálculo: " + str(contagem_lista_outliers_2) + ")\n"
    texto += "------------------------------------------------------\n"
    texto += "Contagem Individual Dados 'Sem Outliers' -> (Pré-Cálculo: " + str(contagem_lista_sem_outliers) + ")\n"
    texto += "Contagem Individual Dados 'Sem Outliers' -> (Pós-Cálculo: " + str(contagem_lista_sem_outliers_2) + ")\n"
    texto += "------------------------------------------------------\n"
    texto += "Contagem dados (Total) Sem Outliers: " + str(sem_outliers) + "\n"
    texto += "Contagem dados (Total) Outliers: " + str(outliers) + "\n"
    texto += "================================================================="
    
    # Salvar em arquivo os resultados dos cálculos Boxplot   
    with open(path_out + '/resultados_boxplots_'+\
                             classe_cor + '_' + nome_imagem[27:35] + '.txt', 'w') as f:
        f.write(texto)
    
    return sem_outliers, outliers
# Chamada de Função para Calcular os valores Finais do segundo Gráfico Boxplot
sem_outliers, outliers=finalizar_calculo_boxplots(sementes, lista_outliers, lista_sem_outliers, boxplot)


#### Processo de Limiarização da Imagem

In [ ]:
## Função para limiarização da imagem (Canal Verde)
def limiarizacao(img_green, th1, th2):
    ### Versão para imagem do Canal Verde (RGB) ###
    # Usar uma cópia da imagem do canal verde (para não alterar a variável original)
    img_gravada_verde = img_green.copy()

    # Chamada do método de Limiarização
    img_limiarizada = fun.threshold(img_gravada_verde, th1, th2)

    # Visualização da imagem
    plt.figure(figsize=[10, 10])
    plt.imshow(img_limiarizada)

    global img_lim
    img_lim = img_limiarizada.copy()
    # Chamada de Função para Binarização da imagem
    img_bin = fun.binarizarImagem(img_lim,th1,th2)
    plt.figure(figsize=[15, 15])

    # Chamada de função para gravar em Disco
    imgLimiarizada = path_out + '/img_limiarizada_bin_' + classe_cor + '_' + nome_imagem
    fun.salvarImagemSegmentadaDisco(img_bin, imgLimiarizada)

    # Chamada de função para upload de arquivos da 'pasta de trabalho' para o Bucket selecionado
    fun.upload_from_bucket('BK-segmented-images-2', url, path_out + '/img_limiarizada_bin_' + classe_cor + '_' + nome_imagem)

    return img_lim, img_bin

## Chamada de função para limiarização da imagem (Canal Verde)
img_lim, img_bin = limiarizacao(img_green, th1, th2)


In [ ]:
# Função para recuperar os pixels de referência
def recuperar_pixels_ref(img_bin, img_orig, path_out, url, nome_imagem):
    #Padronização de nome de variável. Criação de cópia da imagem
    img = img_bin.copy()

    matriz_reconst_bin = fun.reconstruirMatrizPixelsReferencia(img_orig,img, 'IMG_SEGMENTADA')
    plt.imshow(matriz_reconst_bin)

    # Chamada de função para gravar em Disco imagem reconstruida na etapa de limiarização
    imgProcessadaLimiarizacao = path_out + '/imagem_reconstruida_etapa_lim_' + classe_cor + '_' + nome_imagem
    fun.salvarImagemDiscoRGB(matriz_reconst_bin, imgProcessadaLimiarizacao)

    # Chamada de função para upload de arquivos da 'pasta de trabalho' para o Bucket selecionado
    fun.upload_from_bucket('BK-segmented-images-2', url, path_out + '/imagem_reconstruida_etapa_lim_' + classe_cor + '_' + nome_imagem)

    # Padronização da imagem reconstruída
    img = matriz_reconst_bin.copy()

    return img

# Função para recuperar os pixes de referência
img = recuperar_pixels_ref(img_bin, img_orig, path_out, url, nome_imagem)


#### Aplicação do Método de Segmentação - Uso do K-means no espaço de cores RGB

In [ ]:
# Função algoritmo K-Means
def algoritmo_k_means(img):
    num_clusters = 6
    km,labels,img_labels = fun.kmeans(num_clusters,img)

    # Exibir a imagem com todos os labels juntos
    plt.figure(figsize=[15,15])
    plt.imshow(img_labels)
    print(img_labels.shape)

    return km,labels,img_labels, num_clusters

# Chamada de função algoritmo K-Means
km,labels,img_labels, num_clusters = algoritmo_k_means(img)


#### Visualização das imagens original e os labels da Segmentação (mais significativos)

In [ ]:
# Função para exibir os Labels Gerados pelo Algoritmo K-Means
def exibir_labels_k_means(img_labels_closed):
    plt.figure(figsize=[25, 25])
    plt.subplot(1, 6, 1)
    plt.imshow(img_labels_closed[0],'gray')
    plt.title('Label 0')

    plt.subplot(1, 6, 2)
    plt.imshow(img_labels_closed[1],'gray')
    plt.title('Label 1')

    plt.subplot(1, 6, 3)
    plt.imshow(img_labels_closed[2],'gray')
    plt.title('Label 2')

    plt.subplot(1, 6, 4)
    plt.imshow(img_labels_closed[3],'gray')
    plt.title('Label 3')

    plt.subplot(1, 6, 5)
    plt.imshow(img_labels_closed[4],'gray')
    plt.title('Label 4')

    plt.subplot(1, 6, 6)
    plt.imshow(img_labels_closed[5],'gray')
    plt.title('Label 5')

# Chamada Função para exibir rotulos (baseado no número de clusters)
## Exibe os rótulos separados da imagem processada (Cada Cluster do K-Means gera um rótulo)
img_labels_closed = fun.exibeRotulos(num_clusters, img_labels, img)
# Chamada de função para exibir os Labels Gerados pelo Algoritmo K-Means
exibir_labels_k_means(img_labels_closed)

#### Processar os labels segmentados - Matting (Pixel da Imagem Original)

In [21]:
# Função para restaurar os pixels originais da imagem de rederência - Etapa Final Sementação
def restaurar_pixels_kmeans(img_orig,img_labels_closed, coordJanela_1, coord_Janela_2, coordJanela_3, coordJanela_4):
    global matriz_restaurada
    matriz_restaurada =[]
    # Restauração de label 0
    matriz_restaurada_0 = fun.reconstruirMatrizPixelsReferencia(img_orig,img_labels_closed[0], 'IMG_FINAL',\
         coordJanela_1, coord_Janela_2, coordJanela_3, coordJanela_4)
    matriz_restaurada.append(matriz_restaurada_0)
    # Restauração de label 1
    matriz_restaurada_1 = fun.reconstruirMatrizPixelsReferencia(img_orig,img_labels_closed[1], 'IMG_FINAL',\
         coordJanela_1, coord_Janela_2, coordJanela_3, coordJanela_4)
    matriz_restaurada.append(matriz_restaurada_1)
    # Restauração de label 2
    matriz_restaurada_2 = fun.reconstruirMatrizPixelsReferencia(img_orig,img_labels_closed[2], 'IMG_FINAL',\
         coordJanela_1, coord_Janela_2, coordJanela_3, coordJanela_4)
    matriz_restaurada.append(matriz_restaurada_2)
    # Restauração de label 3
    matriz_restaurada_3 = fun.reconstruirMatrizPixelsReferencia(img_orig,img_labels_closed[3], 'IMG_FINAL',\
         coordJanela_1, coord_Janela_2, coordJanela_3, coordJanela_4)
    matriz_restaurada.append(matriz_restaurada_3)
    # Restauração de label 4
    matriz_restaurada_4 = fun.reconstruirMatrizPixelsReferencia(img_orig,img_labels_closed[4], 'IMG_FINAL',\
         coordJanela_1, coord_Janela_2, coordJanela_3, coordJanela_4)
    matriz_restaurada.append(matriz_restaurada_4)
    # Restauração de label 5
    matriz_restaurada_5 = fun.reconstruirMatrizPixelsReferencia(img_orig,img_labels_closed[5], 'IMG_FINAL',\
         coordJanela_1, coord_Janela_2, coordJanela_3, coordJanela_4)
    matriz_restaurada.append(matriz_restaurada_5)

    return matriz_restaurada

# Chamada de função para restaurar os pixels originais da imagem de rederência - Etapa Final Sementação
matriz_restaurada = restaurar_pixels_kmeans(img_orig,img_labels_closed, coordJanela_1,\
                                            coordJanela_2, coordJanela_3, coordJanela_4)

In [ ]:
# Função para exibir as imagens Restauradas - Labels de 0 até 2
def exibir_labels_0_2(matriz_restaurada):
    plt.figure(figsize=[20, 20])
    plt.subplot(1, 3, 1)
    plt.imshow(matriz_restaurada[0])
    plt.title('Segmentada - Label 0')
    plt.subplot(1, 3, 2)
    plt.imshow(matriz_restaurada[1])
    plt.title('Segmentada - Label 1')
    plt.subplot(1, 3, 3)
    plt.imshow(matriz_restaurada[2])
    plt.title('Segmentada - Label 2')

# Chamada de função para exibir as imagens Restauradas - Labels de 0 até 2
exibir_labels_0_2(matriz_restaurada)

In [ ]:
##### Função para exibir as imagens Restauradas - Labels de 3 até 5
def exibir_labels_3_5(matriz_restaurada):
    plt.figure(figsize=[20, 20])
    plt.subplot(1, 3, 1)
    plt.imshow(matriz_restaurada[3])
    plt.title('Segmentada - Label 3')
    plt.subplot(1, 3, 2)
    plt.imshow(matriz_restaurada[4])
    plt.title('Segmentada - Label 4')
    plt.subplot(1, 3, 3)
    plt.imshow(matriz_restaurada[5])
    plt.title('Segmentada - Label 5')

# Chamada de função para exibir as imagens Restauradas - Labels de 3 até 5
exibir_labels_3_5(matriz_restaurada)

#### Ação Supervisionada

In [ ]:
#### global label_significativo

####-------------------------------------------------------####
####----------------(AÇÃO SUPERVISIONADA)------------------####
# Escolha do label mais significativo do algoritmo K-Means
print("Classe em Análise: " + classe_cor + " \nImagem em Análise: " + nome_imagem + "\n")
label_significativo = int(input("Digite o valor do Label mais significativo (K-Means): "))
####-------------------------------------------------------####

In [25]:
# Função para salvar as imagens Restauradas - Todos os Labels
def salvar_img_restauradas(matriz_restaurada, path_out_labels, url, nome_imagem):
    # Chamada de função para upload de arquivos da 'pasta de trabalho' para o Bucket selecionado
    fun.salvarImagemDiscoRGB(matriz_restaurada[0], path_out_labels + '/img_segmentada_restaurada_0_' + nome_imagem)
    fun.upload_from_bucket('BK-segmented-images-2', url, path_out_labels + '/img_segmentada_restaurada_0_' + nome_imagem)
    fun.salvarImagemDiscoRGB(matriz_restaurada[1], path_out_labels + '/img_segmentada_restaurada_1_' + nome_imagem)
    fun.upload_from_bucket('BK-segmented-images-2', url, path_out_labels + '/img_segmentada_restaurada_1_' + nome_imagem)
    fun.salvarImagemDiscoRGB(matriz_restaurada[2], path_out_labels + '/img_segmentada_restaurada_2_' + nome_imagem)
    fun.upload_from_bucket('BK-segmented-images-2', url, path_out_labels + '/img_segmentada_restaurada_2_' + nome_imagem)
    fun.salvarImagemDiscoRGB(matriz_restaurada[3], path_out_labels + '/img_segmentada_restaurada_3_' + nome_imagem)
    fun.upload_from_bucket('BK-segmented-images-2', url, path_out_labels + '/img_segmentada_restaurada_3_' + nome_imagem)
    fun.salvarImagemDiscoRGB(matriz_restaurada[4], path_out_labels + '/img_segmentada_restaurada_4_' + nome_imagem)    
    fun.upload_from_bucket('BK-segmented-images-2', url, path_out_labels + '/img_segmentada_restaurada_4_' + nome_imagem)
    fun.salvarImagemDiscoRGB(matriz_restaurada[5], path_out_labels + '/img_segmentada_restaurada_5_' + nome_imagem)    
    fun.upload_from_bucket('BK-segmented-images-2', url, path_out_labels + '/img_segmentada_restaurada_5_' + nome_imagem)

# Chamada de função para salvar as imagens Restauradas - Todos os Labels
salvar_img_restauradas(matriz_restaurada, path_out_labels, url, nome_imagem)

#### Exibe label escolhido da imagem segmentada correspondente

In [ ]:
# Função para Exibição do label e imagem escolhidos
def exibir_label_imagem_escolhidos(img_labels_closed, url, label_significativo, path_out_labels, nome_imagem):
    img_gravada = img_labels_closed[label_significativo]
     
    matriz_escolhida = cv2.cvtColor(cv2.imread(\
    path_out_labels + 'img_segmentada_restaurada_' + str(label_significativo) + '_' + nome_imagem),\
                                    cv2.COLOR_BGR2RGB)
    
    # Plotar as imagens
    plt.figure(figsize=[20, 20])
    plt.subplot(1, 2, 1)
    plt.imshow(img_gravada)
    plt.title('Label Escolhido')
    plt.subplot(1, 2, 2)
    plt.imshow(matriz_escolhida)
    plt.title('Imagem Segmentada Correspondente')
    
    # Gravar arquivo label escolhido e segmentada correspondente em disco
    plt.savefig(path_out_labels + '/label_escolhido_e_img_segmentada.jpg', format='jpg')
    
    # Gravar arquivo da imagem restaurada do label escolhido
    fun.salvarImagemDiscoRGB(matriz_escolhida, path_out_labels +\
                             '/img_escolhida_label_' + str(label_significativo) + '_' +\
                             classe_cor + '_' + nome_imagem[27:39])
    
    # Function call Upload files function from workspace files to Selected Bucket
    fun.upload_from_bucket('BK-segmented-images-2', url, path_out_labels +\
                             '/img_escolhida_label_' + str(label_significativo) + '_' +\
                             classe_cor + '_' + nome_imagem[27:39])

    return matriz_escolhida, img_gravada

# Chamada de função para Exibição do label e imagem escolhidos
matriz_escolhida, img_gravada = exibir_label_imagem_escolhidos(img_labels_closed, url,\
                                                               label_significativo, path_out_labels, nome_imagem)

#### Suavização (Filtro de Mediana)

In [ ]:
# Função para aplicação do Filtro de Mediana
def filtro_mediana(img, path_out, url, classe_cor, nome_imagem):
    img_suave = cv2.medianBlur(img,3)

    #--------------------------------------------------------------------
    # Plotar as Imagens que serão aplicadas as Métricas MSE, PSNR e SSIM
    #--------------------------------------------------------------------
    plt.figure(figsize=[20,20])
    plt.subplot(121), plt.imshow(img_suave)
    plt.title('Imagem Escolhida Suavizada "Sem filtro"')
    plt.subplot(122), plt.imshow(matriz_escolhida)
    plt.title('Imagem Segmentada Escolhida "com Filro de Mediana"')
    plt.show()

    # Chamada de função para gravar em Disco a imagem suavizada
    fun.salvarImagemSegmentadaDisco (img_suave, path_out + '/img_final_filtro_mediana_classe_' +\
                                     classe_cor + nome_imagem)
    # Chamada de função para upload de arquivos da 'pasta de trabalho' para o Bucket selecionado
    fun.upload_from_bucket('BK-segmented-images-2', url, path_out + '/img_final_filtro_mediana_classe_' +\
                           classe_cor + nome_imagem)

    return img_suave

# ------ Chamada de função para aplicação do Filtro de Mediana -------
img_suave = filtro_mediana(matriz_escolhida, path_out, url, classe_cor, nome_imagem)


#### Métricas Processo de Filtragem (Filtro Mediana)

In [28]:
# Função para calcular métricas processo filtragem
def metricas_filtragem(img_orig, path_out_labels, url, img_suave, nome_imagem, matriz_escolhida):
          
    text = ''
    text+= "----------------------------------\n"
    
    # Imagens para gerar os Histogramas
       #-> Comparação entre as imagens: (1) Imagem Segmentada de entrada (Original) 
       #                                (2) Imagem Segmentada processada (de acordo com a cor: verde, amarela ou marrom)
    img_gray_1 = cv2.cvtColor(cv2.imread(path_in + nome_imagem), cv2.COLOR_BGR2RGB)
    img_gray_2 = cv2.cvtColor(cv2.imread(path_out_labels + 'img_segmentada_restaurada_' +\
                        str(label_significativo) + '_' + nome_imagem), cv2.COLOR_BGR2RGB)
    img_gray_3 = cv2.cvtColor(cv2.imread(path_out + '/img_final_filtro_mediana_classe_' +\
                                         classe_cor + nome_imagem), cv2.COLOR_BGR2RGB)
    
    # Redimensiona os arrays (imagens) para 2 dimensões e transforma em escala de cinza (para os cálculos das métricas)
    # Imagens para Calcular as métricas (MSE, PSNR e SSIM):(1) Imagem Escolhida Suavizada sem filtro            ->(img_1)
    #                                                      (2) Imagem Segmentada Escolhida com Filro de Mediana ->(img_2)
    
    img_1 = rgb2gray(img_orig.reshape(img_orig.shape[0]*img_orig.shape[1], 3))
    img_2 = rgb2gray(img_suave.reshape(img_suave.shape[0]*img_suave.shape[1], 3))

    # Calcular MSE (Mean Squared Error)
    # Imagem Original e Imagem com Filtro de Mediana
    mse = mean_squared_error(img_1, img_2)
    text+="MSE: " + str(mse) + '\n'

    # Peak signal to noise ratio
    # Imagem Escolhida e Imagem Escolhida com Filtro de Mediana
    psnr = peak_signal_noise_ratio(img_1, img_2)
    text+="PSNR: " + str(psnr) + '\n'
       
    ## Calcular o Structural Similarity Index (SSIM) entre duas imagens    
    score = structural_similarity(img_1, img_2, data_range=img_1.max() - img_1.min())
    
    # Imprimir os valores de Score no arquivo (Texto)
    text +="SSIM: {}".format(score)
    text+= "\n----------------------------------"

    # Gravar arquivo de texto com as métricas sementes_calculadas_janela
    with open(path_out + '/metricas_filtragem_'+\
                             classe_cor + '_' + nome_imagem[27:35] + '.txt', 'w') as f:
        f.write(text)
    # Chamada de função para upload de arquivos da 'pasta de trabalho' para o Bucket selecionado
    fun.upload_from_bucket('BK-segmented-images-2', url, path_out + '/metricas_filtragem_'+\
                             classe_cor + '_' + nome_imagem[27:35] + '.txt')
           
    return mse, psnr, score, text, img_gray_1, img_gray_2, img_gray_3

    # Chamada de função para calcular métricas processo filtragem
mse, psnr, score, text, img_gray_1, img_gray_2, img_gray_3 = metricas_filtragem\
        (img_orig, path_out_labels, url, img_suave, nome_imagem, matriz_escolhida)


#### Comparação de Histogramas - Imagem Original e Segmentada

In [ ]:
# Função para comparar histogramas das imagens
def comparar_histogramas(url, path_out, img_gray_1, img_gray_2, coordJanela_1, coordJanela_2, coordJanela_3,coordJanela_4):
    # Considerada a janela (Objeto de Interesse)
    # Criar a Máscara
    mask = np.zeros(img_gray_1.shape[:2], np.uint8)
    mask[coordJanela_1:coordJanela_2, coordJanela_3:coordJanela_4] = 255
    masked_img = cv2.bitwise_and(img_gray_1,img_gray_1,mask = mask)

    # Calcular o Histograma com a Máscara
    hist_mask = cv2.calcHist([img_gray_1],[0],mask,[256],[0,255])
    hist_mask_2 = cv2.calcHist([img_gray_2],[0],mask,[256],[0,255])

    # Exibir o gráfico de Comparação dos Histogramas
    plt.figure(figsize=[15,15])
    plt.subplot(224), plt.plot(hist_mask, color='red', label='Original'),\
                                plt.plot(hist_mask_2, color ='blue', label='Segmentada')
    plt.title('Histogramas')
    plt.legend()
    plt.xlim([0,255])
  
    # Gravar arquivo em disco - Resultado Canal Verde
    plt.savefig(path_out + '/histograma_DSC_0_' + classe_cor + '_' + nome_imagem + '.jpg', format='jpg')
    
    # Chamada de função para upload de arquivos da 'pasta de trabalho' para o Bucket selecionado
    fun.upload_from_bucket('BK-segmented-images-2', url, path_out + '/histograma_DSC_0_' +\
                           classe_cor + '_' + nome_imagem + '.jpg')
    plt.show() 
        
    return mask, masked_img, hist_mask, hist_mask_2

# Chamada de função para comparar histogramas das imagens
mask, masked_img, hist_mask, hist_mask_2 = comparar_histogramas(url, path_out, img_gray_1, img_gray_2,\
     coordJanela_1,coordJanela_2, coordJanela_3,coordJanela_4)


In [ ]:
contador_imagens+=1
# Verifica se o processamento está finalizado
if contador_imagens == number_files:
    print("\n\n ATENÇÃO!! A ÚLTIMA IMAGEM FOI PROCESSADA!! ")
else:
    print("\n\n Contador incrementado - Próxima Imagem à Processar: " + str(contador_imagens))
    print(" ->> Faltam:", str(number_files-contador_imagens))

#### Mapeamento de Variáveis para Armazenamento em Banco de Dados - Mysql

In [31]:
# Função para definição das variáveis o banco de dados - Mysql
def variaveis_bd_mysql():

    # Tabela cad_testes
    nome_teste = nomeTeste
    tipo_teste = tipoTeste

    # Tabela cad_sementes
    descricao_cor_semente = classe_cor
    cor_RGB_semente = corRGBSemente
    dados_sementes = sementes

    # Tabela testes_segmentacao
    data_hora_teste = dataHora
    coordenadas_semente_central = coordenadasSementeCentral
    coordenadas_calc_janela = coordenadasCalcJanela
    total_pixels_janela = totalPixelsJanela
    sementes_calculadas_janela = sementes
    total_sementes_calculadas = totalSementesCalculadas
    erro_estatistico_calculado = erro
    operacao_teste = op
    desvio_padrao_teste = dp
    limiar_1_teste = th1
    limiar_2_teste = th2
    faixa_limiares = range_value
    variancia_teste = var
    img_janela = cv2.imencode('.jpg',janela.copy()) # Codificação da (matriz numpy) para armazenar no banco de dados
    descricao_img_original = nome_imagem
    tipo_img_original = tipoImgOriginal
    largura_img_original = larguraImgOriginal
    altura_img_original =  alturaImgOriginal
    num_canais_img_original = numCanaisImgOriginal
    total_pixels_img_original = totalPixelsImgOriginal
    # Codificação da (matriz numpy) para armazenar no banco de dados
    img_original = cv2.imencode('.jpg',img_orig.copy())
    # Codificação da (matriz numpy) para armazenar no banco de dados 
    img_filtro_mediana = cv2.imencode('.jpg',img_suave.copy())
    # Codificação da (matriz numpy) para armazenar no banco de dados
    img_canal_verde = cv2.imencode('.jpg',img_green.copy())
    # Codificação da (matriz numpy) para armazenar no banco de dados
    img_limiarizada = cv2.imencode('.jpg',img_lim.copy())
    # Codificação da (matriz numpy) para armazenar no banco de dados
    img_processada_limiarizacao = cv2.imencode('.jpg',img_lim.copy()) 
    relatorio_coleta_aut_semente = coleta_aut_semente
    relatorio_dados_estatisticos = dados_estatisticos
    coord_janela_final_obj_interesse = str(param_coord_janela_Obj_int)

    # Tabela rotulos_segmentados
    img_rotulo_segmentado = matriz_restaurada #imgRotuloSegmentado

    return nome_teste, tipo_teste, descricao_cor_semente, cor_RGB_semente, dados_sementes, data_hora_teste, \
    coordenadas_semente_central, coordenadas_calc_janela, total_pixels_janela, sementes_calculadas_janela,\
    total_sementes_calculadas, erro_estatistico_calculado, operacao_teste, desvio_padrao_teste, limiar_1_teste,\
    limiar_2_teste, faixa_limiares, variancia_teste, img_janela, descricao_img_original, tipo_img_original,\
    largura_img_original, altura_img_original, num_canais_img_original, total_pixels_img_original, img_original, \
    img_filtro_mediana, img_canal_verde, img_limiarizada, img_processada_limiarizacao, relatorio_coleta_aut_semente,\
    relatorio_dados_estatisticos, coord_janela_final_obj_interesse, img_rotulo_segmentado

#Chamada de Função para definição das variáveis o banco de dados - Mysql
nome_teste, tipo_teste, descricao_cor_semente, cor_RGB_semente, dados_sementes, data_hora_teste, \
coordenadas_semente_central, coordenadas_calc_janela, total_pixels_janela, sementes_calculadas_janela,\
total_sementes_calculadas, erro_estatistico_calculado, operacao_teste, desvio_padrao_teste, limiar_1_teste,\
limiar_2_teste, faixa_limiares, variancia_teste, img_janela, descricao_img_original, tipo_img_original,\
largura_img_original, altura_img_original, num_canais_img_original, total_pixels_img_original, img_original, \
img_filtro_mediana, img_canal_verde, img_limiarizada, img_processada_limiarizacao, relatorio_coleta_aut_semente,\
relatorio_dados_estatisticos, coord_janela_final_obj_interesse, img_rotulo_segmentado = variaveis_bd_mysql()


In [32]:
# Backup da Pasta de Caracteristicas no Bucket "backup" após o término do processo
#!zip segmentacao_etapa_2 -r ./Segmentacao_Etapa_2 -6
#url = "https://grinqewrovfi.compat.objectstorage.sa-saopaulo-1.oraclecloud.com"
#fun.upload_from_bucket('BK-backup', url, 'segmentacao_etapa_2.zip')

#### Chamada de instruções de Inserção de Dados - Variáveis e Imagens - Resultantes do Processamento

In [33]:
# Função para inserção dos dados processados no Banco de Dados - Mysql
def inserir_dados_processados():
    # Chamada de função para Inserir dados de Cadastro do Teste
    mensagem = fun.inserir_cadTestes(nomeTeste, tipoTeste)
    print(mensagem)

    # Chamada de função para Inserir dados de Cadastro das Sementes
    mensagem = fun.inserir_cadSementes(classe_cor,corRGBSemente, str(sementes))
    print(mensagem)

    # Chamada de função e tratamento da "Tupla" de retorno da consulta do Mysql
    #    para o último ID inserido na tabela "cad_sementes"
    #  - Conversão de Tupla para String
    #  - Retirada de caracteres indesejáveis
    #  - Conversão para inteiro para o último ID
    id = fun.buscaID_cadSementes()
    idCadSemente = str(id[0])
    idCadSemente= idCadSemente.replace("(", "")
    idCadSemente = idCadSemente.replace(",)", "")
    idCadSemente = int(idCadSemente)

    # Chamada de função e tratamento da "Tupla" de retorno da consulta do Mysql
    #    para o último ID inserido na tabela "cad_testes"
    #  - Conversão de Tupla para String
    #  - Retirada de caracteres indesejáveis
    #  - Conversão para inteiro para o último ID
    id = fun.buscaID_cadTestes()
    print(id)
    idCadTeste = str(id[0])
    print(idCadTeste)
    idCadTeste= idCadTeste.replace("(", "")
    idCadTeste = idCadTeste.replace(",)", "")
    idCadTeste = int(idCadTeste)        

    # Chamada de função para Inserir dados gerais do Processamento da Segmentação

    mensagem = fun.inserir_cadSegmentacao(idCadTeste,idCadSemente, dataHora, coordenadas_semente_central, coordenadas_calc_janela, total_pixels_janela, sementes_calculadas_janela,     \
                    total_sementes_calculadas, erro_estatistico_calculado, operacao_teste, desvio_padrao_teste, limiar_1_teste, limiar_2_teste, faixa_limiares, variancia_teste,        \
                    img_janela, descricao_img_original, tipo_img_original, largura_img_original, altura_img_original, num_canais_img_original, total_pixels_img_original, img_original, \
                    img_filtro_mediana, img_canal_verde, img_limiarizada, img_processada_limiarizacao, relatorio_coleta_aut_semente, relatorio_dados_estatisticos, coord_janela_final_obj_interesse)
    print(mensagem)

    # Chamada de função e tratamento da "Tupla" de retorno da consulta do Mysql
    #    para o último ID inserido na tabela "cad_segmentacao"
    #  - Conversão de Tupla para String
    #  - Retirada de caracteres indesejáveis
    #  - Conversão para inteiro para o último ID 
    id = fun.buscaID_cadSegmentacao()
    idCadSegmentacao  = str(id[0])
    idCadSegmentacao = idCadSegmentacao .replace("(", "")
    idCadSegmentacao  = idCadSegmentacao .replace(",)", "")
    idCadSegmentacao  = int(idCadSegmentacao)

    # Chamada de função para Inserir as Imagens do Processamento da Segmentação (Algoritmo K-Means)
    mensagem = fun.inserir_rotulosSegmentados(img_rotulo_segmentado, idCadSegmentacao)
    print(mensagem)

# Chamada de função para inserção dos dados processados no Banco de Dados - Mysql
inserir_dados_processados()
